In [1]:
import rospy
import pickle
import moveit_commander 
from sensor_msgs.msg import JointState
from moveit_msgs.msg import DisplayTrajectory, RobotTrajectory
import random


In [3]:
import moveit_msgs.msg
rospy.init_node('notebook')
state_pub = rospy.Publisher("notebook_state", moveit_msgs.msg.DisplayRobotState, queue_size=1, latch=True)
traj_pub = rospy.Publisher("notebook_traj", moveit_msgs.msg.DisplayTrajectory, queue_size=1, latch=True)

In [4]:
with open("/home/hri25-group3/ros/src/safety_perception/joint_snapshot1.pkl", "rb") as f:
        data = pickle.load(f)

In [5]:
with open("/home/hri25-group3/ros/src/safety_perception/good_pairs2_fast.pkl", "rb") as f:
        plans = pickle.load(f)

In [6]:
data[0]

{'t': 1761213852.0178509,
 'names': ['arm_shoulder_pan_joint',
  'arm_shoulder_lift_joint',
  'arm_elbow_joint',
  'arm_wrist_1_joint',
  'arm_wrist_2_joint',
  'arm_wrist_3_joint'],
 'q': [-3.1371305624591272,
  -2.5746733150877894,
  -1.473508358001709,
  -0.6813049477389832,
  1.5681782960891724,
  -0.3417566458331507]}

In [7]:
def display_state(s):    
    ds = moveit_msgs.msg.DisplayRobotState()
    ds.state.joint_state.name = s['names']
    ds.state.joint_state.position = s['q']
    state_pub.publish(ds)

display_state(data[5])

In [9]:
def display_plan(p):
    dtraj = moveit_msgs.msg.DisplayTrajectory()
    rtraj = moveit_msgs.msg.RobotTrajectory()
    rtraj.joint_trajectory = p.joint_trajectory
    dtraj.trajectory.append(rtraj)
    traj_pub.publish(dtraj)

display_plan(plans[5,0])

In [10]:
move = moveit_commander.MoveGroupCommander('arm')
move.set_max_velocity_scaling_factor(0.5)

move.set_named_target('ready')
move.go()


for i in range (len(data)-1):
    #start
    js1= JointState()
    js1.name=data[i]['names']
    js1.position=data[i]['q']
    move.set_joint_value_target(js1)
    move.set_max_velocity_scaling_factor(0.5)
    move.go()
    j=i+1
    for j in range (len(data)-1):
        #end
        js2= JointState()
        js2.name=data[i]['names']
        js2.position=data[i]['q']
        move.set_joint_value_target(js2)
        move.set_max_velocity_scaling_factor(0.5)
        move.go()
        #go back to start
        move.set_joint_value_target(js1)
        move.set_max_velocity_scaling_factor(0.5)
        move.go()
        
            


[INFO] [1762774583.156762508]: Loading robot model 'cu_ur5e'...
[INFO] [1762774583.157422568]: No root/virtual joint specified in SRDF. Assuming fixed joint
[INFO] [1762774585.084631995]: Ready to take commands for planning group arm.


[WARN] [1762774583.208595722]: Link operator_clearance has visual geometry but no collision geometry. Collision geometry will be left empty. Fix your URDF file by explicitly specifying collision geometry.


MoveItCommanderException: Error setting joint target. Is the target within bounds?

In [16]:
plans.keys()

dict_keys([(0, 1), (1, 0), (0, 2), (2, 0), (0, 3), (3, 0), (0, 4), (4, 0), (0, 5), (5, 0), (6, 0), (7, 0), (8, 0), (1, 2), (2, 1), (1, 3), (3, 1), (1, 4), (4, 1), (1, 5), (5, 1), (6, 1), (7, 1), (1, 8), (8, 1), (3, 2), (2, 4), (4, 2), (2, 5), (5, 2), (6, 2), (2, 7), (7, 2), (8, 2), (4, 5), (5, 4), (6, 4), (4, 7), (7, 4), (8, 4), (6, 5), (7, 5), (8, 5), (7, 6), (8, 6), (8, 7)])

In [20]:
move = moveit_commander.MoveGroupCommander('arm')
move.set_max_velocity_scaling_factor(0.1)

start= data[0]

js1= JointState()
js1.name= start['names']
js1.position=start['q']



[INFO] [1762773235.009626494]: Ready to take commands for planning group arm.


In [21]:
move.set_joint_value_target(js1)# go to start 0 position

move.go()

[INFO] [1762773240.204751716]: ABORTED: CONTROL_FAILED


False

In [41]:
k0 = [k for k in plans.keys() if isinstance(k, tuple) and len(k) >= 2 and k[0] == 1]
print(k0)




[(1, 0), (1, 2), (1, 3), (1, 4), (1, 5), (1, 8)]


In [42]:
if not k0:
    raise ValueError("No keys starting with 0 found.")

key = random.choice(k0)          # e.g., (0, 5)
plan = plans[key]                # access the plan
print("Picked key:", key)

endpoint=key[1]
#print(endpoint)

plans[key]


Picked key: (1, 5)


joint_trajectory: 
  header: 
    seq: 0
    stamp: 
      secs: 0
      nsecs:         0
    frame_id: "base"
  joint_names: 
    - arm_shoulder_pan_joint
    - arm_shoulder_lift_joint
    - arm_elbow_joint
    - arm_wrist_1_joint
    - arm_wrist_2_joint
    - arm_wrist_3_joint
  points: 
    - 
      positions: [-2.65084678331484, -2.4780293903746546, -0.6429926753044128, -1.6183625660338343, 1.5681655406951904, -0.3416479269610804]
      velocities: [-0.029705767541205224, 0.010107850721172854, -0.06283185307179587, 0.054942283151547776, 0.0001138416372092243, -8.059946562658008e-06]
      accelerations: [-2.970576754120522, 1.0107850721172853, -6.283185307179585, 5.494228315154777, 0.011384163720922429, -0.0008059946562658007]
      effort: []
      time_from_start: 
        secs: 0
        nsecs:         0
    - 
      positions: [-2.665699667085443, -2.472975465014068, -0.6744086018403107, -1.5908914244580603, 1.5682224615137947, -0.3416519569343617]
      velocities: [-0.2970576

In [43]:
rtraj.joint_trajectory = plans[key].joint_trajectory


In [44]:
jt = rtraj.joint_trajectory
assert len(jt.points) > 0, "Chosen trajectory has no points."

# go to first waypoint so controllers don't abort
start_map = dict(zip(jt.joint_names, jt.points[0].positions))
move.set_joint_value_target(start_map)
ok = move.go(wait=True); move.stop()
print("Moved to start:", ok)

# execute the trajectory
ok = move.execute(rtraj, wait=True); move.stop()
print("Executed trajectory:", ok)


[INFO] [1762773693.710934252]: ABORTED: CONTROL_FAILED
Moved to start: False
[INFO] [1762773693.714344806]: ABORTED: CONTROL_FAILED
Executed trajectory: False


In [17]:
def has_outgoing(plans, node):
    # True if there exists any key (node, k) in plans
    return any(k[0] == node for k in plans)

In [21]:
def pick_edge_with_future(plans, i, avoid_edge=None):
    """Pick (i->j) preferring j that has outgoing edges. Avoid a specific edge if given."""
    outs = [k for k in plans if k[0] == i and k != avoid_edge]
    if not outs:
        return None
    # prefer edges whose destination isn't a dead-end
    good = [k for k in outs if has_outgoing(plans, k[1])]
    return random.choice(good if good else outs)[1]

In [22]:
cur_idx = 0

In [112]:
next_idx = pick_edge_with_future(plans, cur_idx)
display_state(data[next_idx])
# TODO: make this next line execution and check for success before going on
display_plan(plans[cur_idx,next_idx])
move.set_max_velocity_scaling_factor(0.5)
move.execute(plans[cur_idx,next_idx],wait=True)
cur_idx = next_idx

[INFO] [1762777546.602907697]: ABORTED: CONTROL_FAILED


In [113]:
r = rospy.Rate(0.7)
while not rospy.is_shutdown():
    next_idx = pick_edge_with_future(plans, cur_idx)
    display_state(data[next_idx])
    # TODO: make this next line execution and check for success before going on
    display_plan(plans[cur_idx,next_idx])
    move.set_max_velocity_scaling_factor(0.5)
    move.execute(plans[cur_idx,next_idx],wait=True)
    print("Current index: ",cur_idx)
    print("Next index: ", next_idx)

    cur_idx = next_idx
    r.sleep()

[INFO] [1762777548.110855967]: ABORTED: CONTROL_FAILED
Current index:  5
Next index:  2
[INFO] [1762777549.538481031]: ABORTED: CONTROL_FAILED
Current index:  2
Next index:  5
[INFO] [1762777550.968514097]: ABORTED: CONTROL_FAILED
Current index:  5
Next index:  4
[INFO] [1762777552.396885928]: ABORTED: CONTROL_FAILED
Current index:  4
Next index:  1
[INFO] [1762777553.824936330]: ABORTED: CONTROL_FAILED
Current index:  1
Next index:  4
[INFO] [1762777555.253119508]: ABORTED: CONTROL_FAILED
Current index:  4
Next index:  1
[INFO] [1762777556.682409063]: ABORTED: CONTROL_FAILED
Current index:  1
Next index:  0
[INFO] [1762777558.110787773]: ABORTED: CONTROL_FAILED
Current index:  0
Next index:  1
[INFO] [1762777559.538606552]: ABORTED: CONTROL_FAILED
Current index:  1
Next index:  3
[INFO] [1762777560.966359392]: ABORTED: CONTROL_FAILED
Current index:  3
Next index:  2
[INFO] [1762777562.396697235]: ABORTED: CONTROL_FAILED
Current index:  2
Next index:  7
[INFO] [1762777563.825378733]: A

KeyboardInterrupt: 

In [86]:
# choose from current index (e.g., cur_idx = 0)
key = pick_edge_with_future(plans, cur_idx)
if key is None:
    raise ValueError(f"No outgoing trajectories from {cur_idx}")

rtraj = plans[key]  # RobotTrajectory

# your existing execution snippet:
jt = rtraj.joint_trajectory
if not jt.points:
    raise ValueError("Chosen trajectory has no points.")

start_map = dict(zip(jt.joint_names, jt.points[0].positions))
move.set_joint_value_target(start_map)
ok = move.go(wait=True); move.stop()
print("Moved to start:", ok)

ok = move.execute(rtraj, wait=True); move.stop()
print("Executed trajectory:", ok)

# update to the new endpoint
cur_idx = key[1]
print("Now at index:", cur_idx)


ValueError: No outgoing trajectories from 8